In [23]:
import random
import numpy as np

class RedNeuronalTicTacToe:
    def __init__(self):
        # Inicialización de la red neuronal
        self.entrada = np.zeros(9)  # 9 neuronas en la capa de entrada
        self.salida = np.zeros(9)   # 9 neuronas en la capa de salida
        self.capa_oculta = np.zeros(9)  # 9 neuronas en la capa oculta
        self.pesos_entrada_oculta = np.random.uniform(-1, 1, (9, 9))  # Pesos de la capa de entrada a la capa oculta
        self.pesos_oculta_salida = np.random.uniform(-1, 1, (9, 9))  # Pesos de la capa oculta a la capa de salida
        self.tasa_aprendizaje = 0.1  # Tasa de aprendizaje
        self.episodios = 10000  # Número de episodios para entrenar

    def activacion(self, x):
        # Función de activación sigmoide
        return 1 / (1 + np.exp(-x))

    def derivada_activacion(self, x):
        # Derivada de la sigmoide
        return x * (1 - x)

    def actualizar_entrada(self, estado):
        # Actualiza la capa de entrada (estado actual del tablero)
        self.entrada = np.array(estado)

    def predecir(self):
        # Propagación hacia adelante (feedforward)
        self.capa_oculta = self.activacion(np.dot(self.entrada, self.pesos_entrada_oculta))
        self.salida = self.activacion(np.dot(self.capa_oculta, self.pesos_oculta_salida))

    def entrenar(self, estado, salida_esperada):
        # Entrenamiento básico con retropropagación simplificada
        self.actualizar_entrada(estado)
        self.predecir()

        # Error en la capa de salida
        error_salida = salida_esperada - self.salida
        delta_salida = error_salida * self.derivada_activacion(self.salida)

        # Error en la capa oculta
        error_oculta = delta_salida.dot(self.pesos_oculta_salida.T)
        delta_oculta = error_oculta * self.derivada_activacion(self.capa_oculta)

        # Actualizar pesos
        self.pesos_oculta_salida += self.capa_oculta.T.dot(delta_salida) * self.tasa_aprendizaje
        self.pesos_entrada_oculta += self.entrada.T.dot(delta_oculta) * self.tasa_aprendizaje

    def jugar(self, tablero):
        # El jugador de la red neuronal hace su jugada
        self.actualizar_entrada(tablero)
        self.predecir()

        # Primero, verificar si el jugador (O) tiene una oportunidad de ganar y bloquearla
        jugada_bloqueo = self.buscar_bloqueo(tablero, -1)  # Buscar bloqueo de O
        if jugada_bloqueo != -1:
            return jugada_bloqueo  # Bloquear la jugada de O

        # Intentar ganar si la red tiene una oportunidad
        jugada_ganar = self.buscar_ganar(tablero, 1)  # Buscar si X puede ganar
        if jugada_ganar != -1:
            return jugada_ganar  # Hacer la jugada para ganar

        # Si no hay necesidad de bloquear ni de ganar, jugar de manera aleatoria
        casillas_vacias = [i for i, x in enumerate(tablero) if x == 0]
        jugada = random.choice(casillas_vacias)
        return jugada

    def buscar_bloqueo(self, tablero, jugador):
        # Verifica si hay una línea con dos "jugador" y una casilla vacía
        combinaciones_ganadoras = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
            [0, 4, 8], [2, 4, 6]  # Diagonales
        ]

        for combinacion in combinaciones_ganadoras:
            line = [tablero[i] for i in combinacion]
            if line.count(jugador) == 2 and line.count(0) == 1:
                # Encuentra la casilla vacía y retorna la posición de la jugada
                return combinacion[line.index(0)]
        return -1  # No hay jugada de bloqueo posible

    def buscar_ganar(self, tablero, jugador):
        # Verifica si hay una línea con dos "jugador" y una casilla vacía para ganar
        combinaciones_ganadoras = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
            [0, 4, 8], [2, 4, 6]  # Diagonales
        ]

        for combinacion in combinaciones_ganadoras:
            line = [tablero[i] for i in combinacion]
            if line.count(jugador) == 2 and line.count(0) == 1:
                # Encuentra la casilla vacía y retorna la posición de la jugada
                return combinacion[line.index(0)]
        return -1  # No hay jugada de ganar posible

    def evaluar_tablero(self, tablero):
        # Evaluar el estado del tablero: 1 si X gana, -1 si O gana, 0 si sigue
        combinaciones_ganadoras = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
            [0, 4, 8], [2, 4, 6]  # Diagonales
        ]
        for combinacion in combinaciones_ganadoras:
            if tablero[combinacion[0]] == tablero[combinacion[1]] == tablero[combinacion[2]] != 0:
                return tablero[combinacion[0]]  # 1 o -1 dependiendo de quién ganó
        return 0  # No hay ganador

    def jugar_una_partida(self):
        # Juego completo con la red neuronal contra un jugador (el jugador 2)
        tablero = [0] * 9  # 0 significa vacío, 1 es X, -1 es O
        jugador = 1  # Empieza el jugador X

        while True:
            if jugador == 1:  # Jugada de la red neuronal
                jugada = self.jugar(tablero)
                if tablero[jugada] == 0:  # Asegurarse que la casilla esté libre
                    tablero[jugada] = 1
                    print(f"Red neuronal juega en la casilla {jugada}")
            else:  # Jugada del jugador (humanos juegan como O)
                jugada = int(input("Jugador O, ingresa tu jugada (0-8): "))
                if tablero[jugada] == 0:
                    tablero[jugada] = -1
                else:
                    print("Casilla ocupada. Intenta nuevamente.")
                    continue

            # Mostrar el estado del tablero
            self.mostrar_tablero(tablero)

            # Verificar si hay ganador
            resultado = self.evaluar_tablero(tablero)
            if resultado == 1:
                print("¡La red neuronal ha ganado!")
                break
            elif resultado == -1:
                print("¡El jugador O ha ganado!")
                break
            elif 0 not in tablero:
                print("¡Empate!")
                break

            # Cambiar turno
            jugador *= -1

    def mostrar_tablero(self, tablero):
        # Mostrar el tablero de forma legible
        simbolos = {1: "X", -1: "O", 0: "-"}
        print("\nTablero:")
        for i in range(0, 9, 3):
            print(f"{simbolos[tablero[i]]} | {simbolos[tablero[i+1]]} | {simbolos[tablero[i+2]]}")
            if i < 6:
                print("--+---+--")
        print("\n")


# Crear la red neuronal
red = RedNeuronalTicTacToe()

# Iniciar una partida
red.jugar_una_partida()


Red neuronal juega en la casilla 8

Tablero:
- | - | -
--+---+--
- | - | -
--+---+--
- | - | X


Jugador O, ingresa tu jugada (0-8): 0

Tablero:
O | - | -
--+---+--
- | - | -
--+---+--
- | - | X


Red neuronal juega en la casilla 3

Tablero:
O | - | -
--+---+--
X | - | -
--+---+--
- | - | X


Jugador O, ingresa tu jugada (0-8): 1

Tablero:
O | O | -
--+---+--
X | - | -
--+---+--
- | - | X


Red neuronal juega en la casilla 2

Tablero:
O | O | X
--+---+--
X | - | -
--+---+--
- | - | X


Jugador O, ingresa tu jugada (0-8): 6

Tablero:
O | O | X
--+---+--
X | - | -
--+---+--
O | - | X


Red neuronal juega en la casilla 5

Tablero:
O | O | X
--+---+--
X | - | X
--+---+--
O | - | X


¡La red neuronal ha ganado!
